In [2]:
import numpy as np
import matplotlib.pyplot as plt
import progressbar

np.set_printoptions(precision=3, suppress=True)

## Modelo Físico

https://youtu.be/h-b_6a9xwpA

In [17]:
# Condiciones de prueba: masas colgando verticalmente sin tensión T2
L, L1, L2, L3  = 4, 4, 4, 4  # Longitudes
W1, W2         = 10, 20      # Pesos

#g=9.81

In [18]:
# 1.a (10 puntos)
# 
# Establezca correctamente los valores numéricos de las condiciones
# iniciales de prueba para una  configuración de 2 masas colgando 
# verticalmente con la tensión T2 = 0

# Puede trabajar con 
#    y = [x0,...x8] 
# o con 
#    y=[x1, ... , x9] 
# siguiendo la notación del libro (eq 8.10)

x0 = 1.0 #sin(th1)
x1 = 0 #sin(th2)
x2 = 1 #sin(th3)
x3 = 0 #cos(th1)
x4 = 1 #cos(th2)
x5 = 0 #cos(th3)
x6 = W1 #T1
x7 = 0 #T2
x8 = W2 #T3

y = [x0,x1,x2,x3,x4,x5,x6,x7,x8]

In [19]:
# 1.b (10 puntos)
#
# Escriba las nueve funciones f_i(y)
def f_0(y):
    return L1*y[3]+L2*y[4]+L3*y[5]-L

def f_1(y):
    return L1*y[0]+L2*y[1]-L3*y[2]
    
def f_2(y):
    return y[6]*y[0]-y[7]*y[1]-W1        # algo con longitudes, senos y cosenos, por ejemplo.

def f_3(y):
    return y[6]*y[3]-y[7]*y[4]             

def f_4(y):
    return y[7]*y[1]+y[8]*y[2]-W2       # algo con tensiones, pesos, senos y cosenos, por ejemplo.

def f_5(y):
    return y[7]*y[4]-y[8]*y[5]

def f_6(y):
    return y[0]**2+y[3]**2-1

def f_7(y):
    return y[1]**2+y[4]**2-1

def f_8(y):
    return y[2]**2+y[5]**2-1

In [20]:
# 1.c (10 puntos)
#
# Escriba una función f_vec(y) que retorne la eq. 8.12

def f_vec(y):
    return np.array([f_0(y),f_1(y),f_2(y),f_3(y),f_4(y),f_5(y),f_6(y),f_7(y),f_8(y)])

In [21]:
# 1.d (10 puntos)
# 
# Verifique que con nuestras condiciones de prueba, 
# (masas colgando verticalmente),
# se cumple f_vec(y) = 0

print(f_vec(y))

[0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [22]:
# 1.e (10 puntos)
# 
# implemente la función de derivada parcial para derivar f_i(y) 
# respecto a x_j, evaluada en y.

def derivadaparc(fi,xj,y):
    DeltaX_j = 0.00001
    newvec=np.copy(y)
    for i in range(9):
        if i==xj:
            newvec[i]=newvec[i]+DeltaX_j
    diff=(f_vec(newvec)[fi]-f_vec(y)[fi])/ DeltaX_j 
    
    
    
    # Otras cosas
    return diff # algo muy bonito usando la ecuación 8.18

derivadaparc(1,2,y)

-4.000000000026205

In [23]:
# 1.f (10 puntos)
# 
# Usando la función de derivadas, implemente F como sugiere el libro
# y calcule el Jacobiano como la inversa de F.
def buildjacobian(ynew):
    F = np.zeros([9,9])
    for i in range(9):
        for j in range(9):
            F[i,j]=derivadaparc(i,j,ynew)
    return (np.linalg.inv(F))
        # F[i,j] = derivada parcial de f_i respecto a x_j evaluada en y.
print(buildjacobian(y))
#=np.linalg.inv(F)
#print(jacobiano)

[[ -0.     -0.      0.     -0.     -0.     -0.      0.5     0.     -0.   ]
 [  0.      0.25    0.      0.      0.      0.     -0.5    -0.      0.5  ]
 [ -0.     -0.     -0.      0.      0.      0.      0.      0.      0.5  ]
 [  0.167   0.      0.      0.033   0.      0.033  -0.     -0.333   0.   ]
 [ -0.     -0.     -0.     -0.     -0.     -0.      0.      0.5    -0.   ]
 [  0.083   0.      0.     -0.033   0.     -0.033  -0.     -0.167   0.   ]
 [  0.      0.      1.      0.      0.      0.     -5.     -0.      0.   ]
 [  1.667   0.      0.     -0.667   0.      0.333  -0.     -3.333   0.   ]
 [  0.      0.      0.     -0.      1.     -0.     -0.     -0.    -10.   ]]


$$ \Delta y = J \cdot f(y)$$

In [24]:
# 1.g (10 puntos)
#
# aquí va el delta y.
jacobiano=buildjacobian(y)
deltay=np.matmul(-jacobiano,f_vec(y))
print(deltay)

[0. 0. 0. 0. 0. 0. 0. 0. 0.]


$$ y_{new}=y+\Delta y$$

In [25]:
# 1.h (10 puntos)
#
# Calcule y_new a partir de y y delta y
ynew=y+deltay
print(ynew)

[ 1.  0.  1.  0.  1.  0. 10.  0. 20.]


In [26]:
# 1.j (50 puntos si todo funciona bien) 
#
# Escriba una función que realice varias iteraciones
# delta y = - Jacobiano * f(y)
# y_new = y + delta y
# y = y_new
# y se detenga cuando el valor absoluto de todos 
# los nueve elementos de f(y) sean menores que epsilon
# Imprima el número de pasos realizados

epsilon = 1.e-6

def iterate(y):
    n=0
    bar = progressbar.ProgressBar(max_value=100)
    Y=np.copy(y)
    #print(Y)
    condicion=False
    while(condicion==False or n<100):
        comparable=np.abs(f_vec(Y))
        #print(np.max(comparable))
        if np.max(comparable)<epsilon:
            condicion=True
            break
        jacobiano=buildjacobian(Y)
        delta_y= np.matmul(-jacobiano,f_vec(Y))
        y_new = Y + delta_y
        Y=y_new
        #print(Y)
        n=n+1
        bar.update(n)
        #print(n)
    return(n,y,np.asarray(f_vec(Y)))
    

In [27]:
print(iterate(y))

100% (100 of 100) |######################| Elapsed Time: 0:00:00 ETA:  00:00:00


(0, [1.0, 0, 1, 0, 1, 0, 10, 0, 20], array([0., 0., 0., 0., 0., 0., 0., 0., 0.]))


# Diferentes condiciones iniciales


In [28]:
L, L1, L2, L3  = 3, 4, 4, 8  # Longitudes
W1, W2         = 1, 2      # Pesos

#g=9.81


x00 = 0.972 #sin(th1)
x11 = 0.934 #sin(th2)
x22 = 0.54 #sin(th3)
x33 = 0.234 #cos(th1)
x44 = -0.354#cos(th2)
x55 = -0.840 #cos(th3)
x66 = W1 #T1
x77 = 0 #T2
x88 = W2 #T3

y2 = [x00,x11,x22,x33,x44,x55,x66,x77,x88]

In [29]:
f_vec(y)

array([ 1., -4.,  9.,  0., 18.,  0.,  0.,  0.,  0.])

In [30]:
print(iterate(y2))

N/A% (0 of 100) |                        | Elapsed Time: 0:00:00 ETA:  --:--:--

(6, [0.972, 0.934, 0.54, 0.234, -0.354, -0.84, 1, 0, 2], array([ 0.,  0., -0.,  0., -0., -0.,  0.,  0.,  0.]))
